In [1]:
#!pip install python-dotenv

# TODO
- Odd one out with numbers of answers
- [urgent] Word topic matching is very wrong format -> do numbers make sense here?
- There should be free text answers!
- [urgent] for gap text, implement free text answers more clearly. sometimes the gap number repeats
  - test the free text answer formatting here
  - sometimes provides correct answers in the correct order
- code the level upgrade in python logic
- code the level assessment somehow -> there should be a way to input the language level

In [1]:
import os

import dotenv
dotenv.load_dotenv()

import sys
sys.path.append('..')
import temimbo

import importlib
importlib.reload(temimbo)

<module 'temimbo' from 'c:\\Users\\franz\\OneDrive\\Desktop\\BA\\BA\\LanguageTeacher\\notebooks\\..\\temimbo\\__init__.py'>

In [2]:
connector_openai = temimbo.ConnectorOpenAI(
    openai_key = os.getenv('OPENAI_KEY')
)
tg = temimbo.TaskGenerator(connector_llm = connector_openai)
evaluator = temimbo.AnswerEvaluator(connector_llm = connector_openai)


ui = temimbo.UserInterface()
db = temimbo.DatabaseClientLocalFile(connection_string='./database')
formater = temimbo.Formater()



In [3]:
profile = await db.load_profile(id='Franzy')
domain = await ui.choose_domain(domain='vocabulary')
task_type = await ui.choose_task_type(task_type='multiple_choice')
level, training_goals_subset = await tg.incorporate_profile_in_task(profile, domain)

In [4]:
prompt = await tg.generate_prompt(
    level = level,
    training_goals_subset = training_goals_subset,
    domain = domain,
    task_type = task_type,
)
print(prompt)

Pretend you are a academic english teacher. Generate a task description, and the task according to the format of the examples. No other text or tasks.
Generate a multiple choice with given answer possibilities, there can be multiple correct answers. Generate only one task with four answer possibilities. Generate it for the area of grammar teaching, targeting on simple past.
Adjust the task dificulty to language level C1 (Can understand a wide range of demanding, longer texts, and recognise implicit meaning. Can express him/herself fluently and spontaneously without much obvious searching for expressions. Can use language flexibly and effectively for social, academic and professional purposes. Can produce clear, well-structured, detailed text on complex subjects, showing controlled use of organisational patterns, connectors and cohesive devices).

The following are 2 examples of how the task should be generated with output format and style:

 Select the correct answers:
I am very happy 

In [8]:
raw_output = await tg.generate_task(prompt)
print(raw_output)

Identify the correct form of the verb in the simple past tense:

John _______ his homework before going to bed.
a) finished
b) finish
c) finishes
d) will finish


In [6]:
formatted_output_task = await formater.output_task_formatting(raw_output)
user_answer = await ui.answer_task(formatted_output_task, answer="b)")
formated_user_answer = await formater.learner_answer_formatting(user_answer)


In [7]:
######################
# This is what it will actually looks like
NL_feedback, correctness, training_goals = await evaluator.evaluate_learner_answer(
    domain = domain,
    formatted_output_task = formatted_output_task,
    formated_user_answer = formated_user_answer,
    task_type = task_type,
)
print('NL_feedback =', NL_feedback)
print('correctness =', correctness)
print('training_goals =', training_goals)

NL_feedback = I'm sorry, but your answer is incorrect. The correct answer is 'a) bend them'. The verb 'like' calls for the base form of the verb 'bend'. So it should be 'I like to bend them'. You should practice more on verb forms and pay attention to the verb patterns. Keep up the good work and try again!
correctness = wrong
training_goals = No training goals.


In [11]:
# While the parsing is not implemented, this function will not work
#new_profile = await evaluator.update_learner_profile(training_goals, profile)
#print(f'Old profile:\n{profile}\n\nNew profile:\n{new_profile}')

In [9]:
# This overwrites the original profile
# For testing purposes, it's easier to just comment it out
# await db.save_profile(new_profile)